In [14]:
!git lfs install

Updated Git hooks.
Git LFS initialized.


In [15]:
!cd hermes-function-calling-v1
!git lfs pull

In [17]:
from datasets import load_dataset, load_from_disk
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
import json
import matplotlib.pyplot as plt
import seaborn as sns
import re
from tqdm import tqdm
import networkx as nx

In [26]:
!ls hermes-function-calling-v1/

func-calling.json		 json-mode-agentic.json
func-calling-singleturn.json	 json-mode-singleturn.json
glaive-function-calling-5k.json  README.md


In [20]:
dataset = load_dataset('json', data_files='hermes-function-calling-v1/func-calling.json')

Generating train split: 0 examples [00:00, ? examples/s]

In [22]:
dataset['train']

Dataset({
    features: ['id', 'conversations', 'category', 'subcategory', 'task'],
    num_rows: 1893
})

In [24]:
dataset['train'][123].keys()

dict_keys(['id', 'conversations', 'category', 'subcategory', 'task'])

In [25]:
dataset['train'][123]

{'id': '3bac4895-1a36-4380-8ef7-c36abfcc4ba4',
 'conversations': [{'from': 'system',
   'value': "You are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.\n<tools>\n[{'type': 'function', 'function': {'name': 'scale_up_cluster', 'description': 'Scales up the number of compute nodes in an HPC cluster.', 'parameters': {'type': 'object', 'properties': {'cluster_name': {'type': 'string', 'description': 'The name of the HPC cluster to scale up.'}, 'node_type': {'type': 'string', 'description': 'The type of compute node to add to the cluster.'}, 'additional_nodes': {'type': 'integer', 'description': 'The number of additional compute nodes to add to the cluster.'}}, 'required': ['cluster_name', 'node_type', 'additional_nodes']}}}, {'type': 'function', 'function': {'name': 'scale_down_cluster', 'description': '

In [ ]:
dataset = load_dataset('json', data_files='hermes-function-calling-v1/func-calling.json')

In [36]:
d['train'][400]

{'id': 'f9b15d86-fb69-4f05-a65e-18f2f7b5369d',
 'conversations': [{'from': 'system',
   'value': "You are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.\n<tools>\n[{'type': 'function', 'function': {'name': 'min_max_normalize', 'description': 'Apply Min-Max normalization to scale features in the dataset to a specified range.', 'parameters': {'type': 'object', 'properties': {'data': {'type': 'string', 'description': 'The path to the dataset file.'}, 'feature_range': {'type': 'object', 'properties': {'min': {'type': 'number', 'description': 'The minimum value of the target range for normalization.'}, 'max': {'type': 'number', 'description': 'The maximum value of the target range for normalization.'}}, 'required': ['min', 'max']}}, 'required': ['data', 'feature_range']}}}, {'type': 'function', 'function'

So we need to address a few things: 

Mapping:
System is fine but we need to change the function and also the XML tag nonsense

Human -> User

GPT -> Assistant

In [37]:
def process_dataset(dataset):
   def replace_terms(data):
       if isinstance(data, str):
           return data.replace('human', 'user').replace('gpt', 'assistant')
       elif isinstance(data, list):
           return [replace_terms(item) for item in data]
       elif isinstance(data, dict):
           return {k: replace_terms(v) for k, v in data.items()}
       return data
   
   return dataset.map(lambda x: {k: replace_terms(v) for k, v in x.items()})

processed_dataset = process_dataset(d)

Map:   0%|          | 0/1893 [00:00<?, ? examples/s]

In [38]:
d = processed_dataset

In [39]:
d['train'][123]

{'id': '936b90da-dbeb-4864-a6e6-28899965265d',
 'conversations': [{'from': 'system',
   'value': "You are a function calling AI model. You are provided with function signatures within <tools> </tools> XML tags. You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug into functions.\n<tools>\n[{'type': 'function', 'function': {'name': 'scale_up_cluster', 'description': 'Scales up the number of compute nodes in an HPC cluster.', 'parameters': {'type': 'object', 'properties': {'cluster_name': {'type': 'string', 'description': 'The name of the HPC cluster to scale up.'}, 'node_type': {'type': 'string', 'description': 'The type of compute node to add to the cluster.'}, 'additional_nodes': {'type': 'integer', 'description': 'The number of additional compute nodes to add to the cluster.'}}, 'required': ['cluster_name', 'node_type', 'additional_nodes']}}}, {'type': 'function', 'function': {'name': 'scale_down_cluster', 'description': '

In [40]:
NEW_SYSTEM_MESSAGE = """You are an expert in composing functions. You are given a question and a set of possible functions.
Based on the question, you will need to make one or more function/tool calls to achieve the purpose.
If none of the function can be used, point it out. If the given question lacks the parameters required by the function,
also point it out. You should only return the function call in tools call sections.
If you decide to invoke any of the function(s), you MUST put it in the format of [func_name1(params_name1=params_value1, params_name2=params_value2...), func_name2(params)]
You SHOULD NOT include any other text in the response.
Here is a list of functions in JSON format that you can invoke."""